In [1]:
import nengo
import nengo_spa as spa
import numpy as np
# import nengo_ocl
import sys, os
import math
import matplotlib.pyplot as plt
%matplotlib inline

from IPython import display
from nengo_gui.ipython import IPythonViz

This is a model that learns to do two tasks: A Choice reaction time task and a Working Memory task. First set global parameters and populate the vocabulary.

In [2]:
s = spa.sym
D = 128  # the dimensionality of the vectors

number_of_learning_trials = 40
trial_length = 3.4  # This is actually for a WMTASK trial and a CRTTASK trial together, they are 1.7 sec each

number_of_running_trials = 60
T = number_of_running_trials * trial_length

rng = np.random.RandomState(0)

symbol_keys = {'ONE', 'TWO', 'THREE', 'NINE', 'ODD', 'EVEN', 'BLACK', 'RED', \
    'G1', 'V1', 'V2', 'WM1', 'RT1', 'RT2', 'AC1', 'PRIM',  'WMTASK', \
    'CRTTASK', 'QUESTION', 'EPISODE', 'CRY', 'REDEEM', 'LAUGH' }
prim_keys = {'V1RT1', 'V1WM1', 'RT2AC1', 'RT2WM1', 'WM1RT1', 'WM1AC1', 'FOCUS', 'DONOTHING'}

all_keys = symbol_keys | prim_keys

#vocab_memory = spa.Vocabulary(dimensions=D,rng=rng, name='all')
vocab_memory = spa.Vocabulary(dimensions=D, name='all')

vocab_memory.populate(";".join(all_keys))

prim_vocab = vocab_memory.create_subset(prim_keys)

# Taskname can be either WMTASK or CRTTASK
# taskname = 'WMTASK'
# Is now set by the simulation
taskname = 'CRTTASK'


/home/hugo/anaconda3/envs/CTN/lib/python3.7/site-packages/nengo_spa/vocabulary.py:173: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=128, M=26, similarity=0.11)
  len(self._key2idx), best_sim))


In [3]:
model = spa.Network(seed=12) # was 14
with model:
    # You can set variable to either s.CRTTASK or s.WMTASK
    if taskname == 'WMTASK':
        task = s.WMTASK
    else:
        task = s.CRTTASK

    # We start defining the buffer slots in which information can
    # be placed:
    # Two visual buffers. We will use vison1 for the digit and vision2 for the color
    vision1 = spa.State(vocab_memory, label='vision1')
    vision2 = spa.State(vocab_memory, label='vision2')

    # This is the buffer where we want the answer
    action = spa.State(vocab_memory, feedback=0.8, feedback_synapse=0.01, label='action')

    # we can place a memory request in memory1, and the answer will be place in memory2
    memory1 = spa.State(vocab_memory, label='memory1')
    memory1a = spa.State(vocab_memory, label='memory1a', feedback = 0.8, feedback_synapse=0.05)
    memory2 = spa.State(vocab_memory, feedback=0.9, feedback_synapse=0.05, label='memory2')
    
    memory1 >> memory1a
    
    # Finally, we have a working memory with one slot
    wm = spa.State(vocab_memory, feedback=0.90, feedback_synapse=0.05, label='wm')

    # A slot for the goal
    goal = spa.State(vocab_memory, label = 'goal')

    
    # A state to store the previous PRIM
    prev_prim = spa.State(vocab_memory, label = 'prev_prim')
    
    # Now bind all the state slots together in one big semantic pointer
   
    combined_state = spa.State(vocab_memory, neurons_per_dimension = 150, label='combined_state')
    
    goal*s.G1 + vision1*s.V1 + vision2*s.V2 + wm*s.WM1 + \
    memory1*s.RT1 + memory2*s.RT2 + prev_prim*s.PRIM >> combined_state
 
    # The prim state will receive the prim to be executed based on the combined_state
    prim = spa.State(prim_vocab, label='prim')

    # We will do supervised learned, so this state will hold the correct action
    correct_action = spa.State(prim_vocab, label='correct action')
    
    # This state will be used to calculated the error
    error = spa.State(prim_vocab, label='error')
    prim - correct_action >> error

    # Randomize decoders between ensembles and output
    for ens in combined_state.all_ensembles:
        con = nengo.Connection(ens, prim.input, function=lambda t: [0]*D,
                               learning_rule_type=nengo.PES(learning_rate=1e-5)) # was 1e-4
        nengo.Connection(error.output, con.learning_rule)
    #    ens.intercepts= nengo.dists.Uniform(0.15, 0.2)

    

    def stimuli_for_trial(trial, i):
        numbers = ['ONE','TWO','THREE','NINE']
        return numbers[(trial * 3 + i) % numbers.__len__()]
    
    def event_queue(t):
        trial_number = math.floor( t / trial_length)
        t_in_trial = t - trial_number * trial_length
        if t_in_trial < 1.7:
            taskname == 'WMTASK'
            if trial_number < number_of_learning_trials:
                if 0.0 < t_in_trial < 0.3:
                    return stimuli_for_trial(trial_number,0), 'BLACK', 'V1WM1', 0, s.WMTASK
                elif 0.5 < t_in_trial < 0.8:
                    return stimuli_for_trial(trial_number,1), 'BLACK', 'V1WM1', 0, s.WMTASK    
                elif 1.0 < t_in_trial < 1.3:
                    return 'QUESTION', 'RED', 'WM1RT1', 0, s.WMTASK
                elif 1.3 < t_in_trial < 1.6:
                    return 'QUESTION', 'RED', 'RT2AC1', 0, s.WMTASK
                else:
                    return '0', '0', 'DONOTHING', 0, s.WMTASK
            else:
                if 0.0 < t_in_trial < 0.3:
                    return stimuli_for_trial(trial_number,0), 'BLACK', 'V1WM1', -1, s.WMTASK
                elif 0.5 < t_in_trial < 0.8:
                    return stimuli_for_trial(trial_number,1), 'BLACK', 'V1WM1', -1, s.WMTASK   
                elif 1.0 < t_in_trial < 1.3:
                    return 'QUESTION', 'RED', 'WM1RT1', -1, s.WMTASK
                elif 1.3 < t_in_trial < 1.6:
                    return 'QUESTION', 'RED', 'RT2AC1', -1, s.WMTASK
                else:
                    return '0', '0', 'DONOTHING', -1, s.WMTASK
        else:
            taskname == 'CRTTASK'
            t_in_trial -= 1.7
            if trial_number < number_of_learning_trials:
                if 0.0 < t_in_trial < 0.3:
                    return stimuli_for_trial(trial_number,0), 'BLACK', 'FOCUS', 0, s.CRTTASK
                elif 0.5 < t_in_trial < 0.8:
                    return stimuli_for_trial(trial_number,1), 'BLACK', 'FOCUS', 0, s.CRTTASK    
                elif 1.0 < t_in_trial < 1.3:
                    return stimuli_for_trial(trial_number,2), 'RED', 'V1RT1', 0, s.CRTTASK
                elif 1.3 < t_in_trial < 1.6:
                    return stimuli_for_trial(trial_number,2), 'RED', 'RT2AC1', 0, s.CRTTASK
                else:
                    return '0', '0', 'DONOTHING', 0, s.CRTTASK
            else:
                if 0.0 < t_in_trial < 0.3:
                    return stimuli_for_trial(trial_number,0), 'BLACK', 'FOCUS', -1, s.CRTTASK
                elif 0.5 < t_in_trial < 0.8:
                    return stimuli_for_trial(trial_number,1), 'BLACK', 'FOCUS', -1, s.CRTTASK    
                elif 1.0 < t_in_trial < 1.3:
                    return stimuli_for_trial(trial_number,2), 'RED', 'V1RT1', -1, s.CRTTASK
                elif 1.3 < t_in_trial < 1.6:
                    return stimuli_for_trial(trial_number,2), 'RED', 'RT2AC1', -1, s.CRTTASK
                else:
                    return '0', '0', 'DONOTHING', -1, s.CRTTASK
            

    def vision_func(t):
        return event_queue(t)[0]
    
    def color_func(t):
        return event_queue(t)[1]
    
    def correct_func(t):
        return event_queue(t)[2]
    
    def task_function(t):
        return event_queue(t)[4]
    
    # Feed the input in the vision1 and vision2
    with spa.Network(label='inputs'):
        vision_input = spa.Transcode(vision_func,output_vocab = vocab_memory)
        color_input = spa.Transcode(color_func,output_vocab = vocab_memory)
        correct_input = spa.Transcode(correct_func, output_vocab = prim_vocab)
        task_input = spa.Transcode(task_function, output_vocab = vocab_memory)

    vision_input >> vision1
    
    color_input >> vision2
    correct_input >> correct_action
    task_input >> goal  
    inhibit = nengo.Node(output = lambda t: event_queue(t)[3])
    inhibit2 = spa.Scalar(label='PRIMSdone')
    for ens in error.all_ensembles:
        nengo.Connection(inhibit, ens.neurons, transform=np.ones((ens.n_neurons, 1)) * 10, synapse=None)
        nengo.Connection(inhibit2.output, ens.neurons, transform=np.ones((ens.n_neurons, 1)) , synapse=None)

    # Definition of the actions
    # There are rules that carry out the actions, and rules that check the
    # conditions. If a condition is satisfied, check is set to YES which
    # is a condition for the actions.
    with spa.Network(label='BG-Thalamus') :
        with spa.ActionSelection() as bg_thalamus:
            # Action rules first
            spa.ifmax( spa.dot(prim, s.V1RT1),
                        vision1 >> memory1)
            spa.ifmax( spa.dot(prim, s.RT2AC1),
                        memory2 >> action)
            spa.ifmax( spa.dot(prim, s.V1WM1),
                        vision1 >> wm)
            spa.ifmax( spa.dot(prim, s.WM1RT1),
                        wm >> memory1)
            spa.ifmax( spa.dot(prim, s.RT2WM1),
                        memory2 >> wm)
            spa.ifmax( spa.dot(prim, s.WM1AC1),
                        wm >> action)
#            spa.ifmax( spa.dot(prim, s.FOCUS),
#                        s.FOCUS >> prev_prim)
            spa.ifmax(0.3,
                        s.EPISODE >> memory1, 10 * memory2 >> memory1)
#            spa.ifmax( 0.35 * spa.dot(memory2, s.EPISODE),
#                        memory2 >> memory1)
#            spa.ifmax(0.3, s.DONOTHING >> prev_prim)
    
    
    with spa.Network(label='Inhibit- BG-Thalamus'):
        with spa.ActionSelection():
            spa.ifmax( spa.dot(correct_action, s.V1WM1) * spa.dot(vision1,wm) ,
                        -1.0 >> inhibit2, s.V1WM1 >> prev_prim)
            spa.ifmax( spa.dot(correct_action, s.WM1RT1) * spa.dot(wm,memory1) ,
                        -1.0 >> inhibit2, s.WM1RT1 >> prev_prim)
            spa.ifmax( spa.dot(correct_action, s.RT2AC1) * spa.dot(memory2,action),
                        -1.0 >> inhibit2, s.RT2AC1 >> prev_prim)
            spa.ifmax( spa.dot(correct_action, s.V1RT1) * spa.dot(vision1, memory1),
                        -1.0 >> inhibit2, s.V1RT1 >> prev_prim)
            spa.ifmax(0.3, 0.0 >> inhibit2)
            
    # A simple memory system
    
    mapping = {
        'ONE' : 'ODD',
        'TWO' : 'EVEN',
        'THREE' : 'ODD',
        'NINE' : 'ODD',
        'EPISODE' : 'CRY',
        'CRY' : 'REDEEM',
        'REDEEM' : 'LAUGH'
    }
    
    memory = spa.WTAAssocMem(
        threshold = 0.3,
        input_vocab = vocab_memory,
        mapping = mapping,
        label = 'memory',
        function = lambda x: x > 0.)
        
    memory1a >> memory
    memory.output >> memory2

Set up some probes

In [4]:
with model:
    p_sel = nengo.Probe(bg_thalamus.thalamus.output, synapse = 0.01, sample_every = 0.01)
    p_util = nengo.Probe(bg_thalamus.bg.input, synapse = 0.01, sample_every = 0.01)
    p_mem = nengo.Probe(memory2.output, synapse = 0.01, sample_every = 0.01)

Run de model

In [5]:
with nengo.Simulator(model) as sim:
    sim.run(T)

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/hugo/anaconda3/envs/CTN/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7de1e4ae4718>", line 2, in <module>
    sim.run(T)
  File "/home/hugo/anaconda3/envs/CTN/lib/python3.7/site-packages/nengo/simulator.py", line 333, in run
    self.run_steps(steps, progress_bar=progress_bar)
  File "/home/hugo/anaconda3/envs/CTN/lib/python3.7/site-packages/nengo/simulator.py", line 357, in run_steps
    self.step()
  File "/home/hugo/anaconda3/envs/CTN/lib/python3.7/site-packages/nengo/simulator.py", line 368, in step
    step_fn()
  File "/home/hugo/anaconda3/envs/CTN/lib/python3.7/site-packages/nengo/builder/operator.py", line 625, in step_dotinc
    Y[...] += A.dot(X)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hugo/anaconda3/envs/CTN/lib/py

KeyboardInterrupt: 

In [16]:
file = open('th-output-2.txt', 'w')
i = 0
for item in sim.data[p_sel]:
    i += 1
    j = 0
    for sub_item in item:
        j += 1
        print(i, j, sub_item, file=file)
#        file.write("%s %s %s\n" % i % j % sub_item)
file.close()

In [17]:
file = open('bg-util-2.txt', 'w')
i = 0
for item in sim.data[p_util]:
    i += 1
    j = 0
    for sub_item in item:
        j += 1
        print(i, j, sub_item, file=file)
#        file.write("%s %s %s\n" % i % j % sub_item)
file.close()

In [22]:
file = open('mem-output-2.txt', 'w')
i = 0
for item in sim.data[p_mem]:
    i += 1
    for entry in vocab_memory.keys():
        print(i, entry, spa.similarity(item, vocab_memory[entry].v)[0], file=file)
#        file.write("%s %s %s\n" % i % j % sub_item)
file.close()